<br>

***

yo

# <span style="color:#0070C0;font-size:35px"><b>Week 4</b></span>
<!-- # <span style="color:#0070C0;font-size:35px"><b>Title</b></span> -->
<!-- # <span style="color:#0070C0">H1</span> -->
<!-- ## <span style="color:#2A9FC4">H2</span> -->
<!-- ### <span style="color:#44AA57">H3</span> -->

<span style="color:#8c8c8c">Patrick Weatherford</span>

[Dark Blue]: <> (#0070C0)
[Light Blue]: <> (#2A9FC4)
[Earth Green]: <> (#44AA57)
[Grey]: <> (#8c8c8c)

<br>

***

# <span style="color:#0070C0">Import Libraries</span>
<!-- # <span style="color:#0070C0;font-size:40px"><b>Title</b></span> -->
<!-- # <span style="color:#0070C0">H1</span> -->
<!-- ## <span style="color:#2A9FC4">H2</span> -->
<!-- ### <span style="color:#44AA57">H3</span> -->

<!-- <span style="color:#8c8c8c">Patrick Weatherford</span> -->

[Dark Blue]: <> (#0070C0)
[Light Blue]: <> (#2A9FC4)
[Earth Green]: <> (#44AA57)
[Grey]: <> (#8c8c8c)

Custom module on github: https://github.com/Hakuna-Patata/BU_MSDS_PTW/blob/main/Python/HakunaPatata.py

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/Othercomputers/My Laptop/OneDrive - Bellevue University/Bellevue_University/DSC 550 - Data Mining'


/content/drive/Othercomputers/My Laptop/OneDrive - Bellevue University/Bellevue_University/DSC 550 - Data Mining


In [ ]:
import os
import sys

home_path = f"C:{os.environ['HOMEPATH']}"
bu_path = f"{home_path}\\OneDrive - Bellevue University\\Bellevue_University"

custom_mod_path = f"{bu_path}\\Python\\Custom_Modules".replace('\\', '/')

sys.path.append(custom_mod_path)

import api_keys
import hakuna_patata

KeyError: ignored

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pprint import pprint
from zipfile import ZipFile
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

# matplotlib default settings
plt.style.use('dark_background')
mpl.rcParams.update({'lines.linewidth': 3})
mpl.rcParams.update({'axes.labelsize': 14})
mpl.rcParams.update({'axes.titlesize': 16})
mpl.rcParams.update({'axes.titleweight': 'bold'})
mpl.rcParams.update({'figure.autolayout': True})
mpl.rcParams.update(
    {'axes.grid': True, 'grid.color': '#424242', 'grid.linestyle': '--'})

# creation random number generator object
rng_seed = 777
rng = np.random.default_rng(rng_seed)

# pandas dataframe options
pd.set_option('display.max_columns', None)
